<a href="https://colab.research.google.com/github/kozen88/HCV_DM23/blob/main/DataClean%26processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data cleaning and preparation for future training

In [73]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sb
sb.set()

In [91]:
df = pd.read_csv('HCV-Egy-Data.csv', delimiter=',')
print(df)

      Age   Gender  BMI  Fever  Nausea/Vomting  Headache   Diarrhea   \
0       56       1   35      2               1          1          1   
1       46       1   29      1               2          2          1   
2       57       1   33      2               2          2          2   
3       49       2   33      1               2          1          2   
4       59       1   32      1               1          2          1   
...    ...     ...  ...    ...             ...        ...        ...   
1380    44       1   29      1               2          2          2   
1381    55       1   34      1               2          2          1   
1382    42       1   26      2               2          1          1   
1383    52       1   29      2               1          1          2   
1384    55       2   26      1               2          2          2   

      Fatigue & generalized bone ache   Jaundice   Epigastric pain   ...  \
0                                    2          2          

## Data replacement 1
Si è vagliata la possibilità che i valori nel data set che avessero il valore uguale a 5.0 fossero dei possibili valori di default assegnati quando in realtà il dato non è stato misurato o raccolto. Partendo da tale ipotesi si vuole sostituire questi valori con dei valori null e vedere il comportamento dei modelli in tali condizioni.

In [92]:
# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Replace the values equal to 5.0 with null values
df['RNA 12'] = df['RNA 12'].replace(5.0, np.nan)
df['RNA EOT'] = df['RNA EOT'].replace(5.0, np.nan)
df['RNA EF'] = df['RNA EF'].replace(5.0, np.nan)
df['ALT 36'] = df['ALT 36'].replace(5.0, np.nan)
df['ALT 48'] = df['ALT 48'].replace(5.0, np.nan)
df['ALT after 24 w'] = df['ALT after 24 w'].replace(5.0, np.nan)

# Display the modified DataFrame
print("\nModified DataFrame:")
print(df)

Original DataFrame:
      Age   Gender  BMI  Fever  Nausea/Vomting  Headache   Diarrhea   \
0       56       1   35      2               1          1          1   
1       46       1   29      1               2          2          1   
2       57       1   33      2               2          2          2   
3       49       2   33      1               2          1          2   
4       59       1   32      1               1          2          1   
...    ...     ...  ...    ...             ...        ...        ...   
1380    44       1   29      1               2          2          2   
1381    55       1   34      1               2          2          1   
1382    42       1   26      2               2          1          1   
1383    52       1   29      2               1          1          2   
1384    55       2   26      1               2          2          2   

      Fatigue & generalized bone ache   Jaundice   Epigastric pain   ...  \
0                                    2 

In [76]:
df.to_csv("HCV_null_hipotesis.csv", index=False)
df.to_excel("HCV_null_Hip.xlsx", index=False)

## Data replecement 2
Un altra possibile ipotesi di analisi e che i valori 5.0 poichè non rappresentano il minimo del test fatto in quanto il minimo è 12 per il test di Abott sia tutti errori di raccolta dei dati e quindi sotto questa impotesi utlizzeremo un Imputer per sostituirli con il valor medio della rispettiva colonna.

In [77]:
from sklearn.impute import SimpleImputer

# Istanziamo un oggetto Imputer che andrà a sostituire i valori che specificheremo con la media
imputer = SimpleImputer(strategy='mean')

# l'imputer entra in azione sul dataframe
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Display the modified DataFrame
print(df_imputed)

      Age   Gender   BMI  Fever  Nausea/Vomting  Headache   Diarrhea   \
0     56.0     1.0  35.0    2.0             1.0        1.0        1.0   
1     46.0     1.0  29.0    1.0             2.0        2.0        1.0   
2     57.0     1.0  33.0    2.0             2.0        2.0        2.0   
3     49.0     2.0  33.0    1.0             2.0        1.0        2.0   
4     59.0     1.0  32.0    1.0             1.0        2.0        1.0   
...    ...     ...   ...    ...             ...        ...        ...   
1380  44.0     1.0  29.0    1.0             2.0        2.0        2.0   
1381  55.0     1.0  34.0    1.0             2.0        2.0        1.0   
1382  42.0     1.0  26.0    2.0             2.0        1.0        1.0   
1383  52.0     1.0  29.0    2.0             1.0        1.0        2.0   
1384  55.0     2.0  26.0    1.0             2.0        2.0        2.0   

      Fatigue & generalized bone ache   Jaundice   Epigastric pain   ...  \
0                                  2.0        2

Andiamo a creare due file rispettivamente un excell per andare a contrallare il lavoro svolto e un csv per futuri utilizzi senza doverlo ricreare

In [79]:
df_imputed.to_csv("HCV_ImputerMean.csv", index=False)
df_imputed.to_excel("HCV_ImputerMean.xlsx", index=False)

## Replacement 3
In questo caso andremo a lavorare con l'ipotesi che i 5 rappresentino il valore di range basso minimo per il test e lo sostituiremo con il minimo valore per l'rna e con un valore all'interno del range normale per l'alt.

In [93]:
# assicuriamoci che ogni categoria di nullo sia trattata allo stesso modo
missing_values = ['None', 'NA', 'Missing', 'NaN', 'N/A']
df = df.replace(missing_values, np.nan)

# list contenente colonne da dare in pasto all'imputer
columns_to_impute = ['RNA 12', 'RNA EOT', 'RNA EF']

# istanziazione dell'imputer
imputer_rna = SimpleImputer(strategy='constant', fill_value=12)

# mettiamo in azione l'imputer
df[columns_to_impute] = imputer_rna.fit_transform(df[columns_to_impute])

# Display the modified DataFrame
print(df)


      Age   Gender  BMI  Fever  Nausea/Vomting  Headache   Diarrhea   \
0       56       1   35      2               1          1          1   
1       46       1   29      1               2          2          1   
2       57       1   33      2               2          2          2   
3       49       2   33      1               2          1          2   
4       59       1   32      1               1          2          1   
...    ...     ...  ...    ...             ...        ...        ...   
1380    44       1   29      1               2          2          2   
1381    55       1   34      1               2          2          1   
1382    42       1   26      2               2          1          1   
1383    52       1   29      2               1          1          2   
1384    55       2   26      1               2          2          2   

      Fatigue & generalized bone ache   Jaundice   Epigastric pain   ...  \
0                                    2          2          

In [94]:
# list contenente colonne da dare in pasto all'imputer
columns_to_impute = ['ALT 36', 'ALT 48', 'ALT after 24 w']

# istanziazione dell'imputer
imputer_alt = SimpleImputer(strategy='constant', fill_value=35)

# mettiamo in azione l'imputer
df[columns_to_impute] = imputer_alt.fit_transform(df[columns_to_impute])

# Display the modified DataFrame
print(df)


      Age   Gender  BMI  Fever  Nausea/Vomting  Headache   Diarrhea   \
0       56       1   35      2               1          1          1   
1       46       1   29      1               2          2          1   
2       57       1   33      2               2          2          2   
3       49       2   33      1               2          1          2   
4       59       1   32      1               1          2          1   
...    ...     ...  ...    ...             ...        ...        ...   
1380    44       1   29      1               2          2          2   
1381    55       1   34      1               2          2          1   
1382    42       1   26      2               2          1          1   
1383    52       1   29      2               1          1          2   
1384    55       2   26      1               2          2          2   

      Fatigue & generalized bone ache   Jaundice   Epigastric pain   ...  \
0                                    2          2          

In [95]:
df.to_csv("HCV_rna12_alt35.csv", index=False)
df.to_excel("HCV_rna12_alt35.xlsx", index=False)